In [9]:
import numpy as np
import matplotlib.pyplot as plt
import idx2numpy

In [178]:
#MNIST classification 

Definir abastract class

Definir linear

In [98]:
class linear_layer():

    def __init__(self, input_dimensions, output_dimensions):
        self.w = 2 * np.random.rand(input_dimensions, output_dimensions) - 1 #Genera un array de 1 a -1
        self.b = np.zeros((1, output_dimensions))

    def forward(self, x):
        self.x = x
        return x @ self.w + self.b #Multiplica les 2 arrays
    
    def backward(self, error):
        keep_prop = 0.25
        gamma = 0.003
        m = error.shape[0]
        #self.dW = self.x.T @ error #Lo mateix que la linea de baix, molt important l'ordre
        #self.dW = 1/m * (self.x.T @ error) #Transposem l'error per a multiplicar correctament les matrius (teoria a la llibreta), dW es el valor a canviar a W2
        self.dW = np.zeros(self.w.shape) 
        self.dB = 1/m * error.sum(axis=0)
        return (self.w @ error.T).T
    
    def update_parameters(self, lr):
        self.w = self.w - lr * self.dW
        self.b = self.b - lr * self.dB

x = np.array([[1,0,1,0],[0,1,1,1]])
ll1 = linear_layer(4,2)

ll1.forward(x)

# error = 2* (ypred - yreal)
error = np.array([[ 0.07912738,  0.77109688],[ 0.73532997, -0.13105195]])

ll1.backward(error)

assert ll1.w.shape == ll1.dW.shape

ll1.update_parameters(0.1)



Definir activation layers and input

In [94]:
class ReLu_layer():
    
    def __init__(self):
        pass
    
    def forward(self, x):
        self.Z = np.maximum(0,x)
        return self.Z
    
    def backward(self, error): #L'error es la derivada de ReLu
        return error * (self.Z > 0)
    
    def update_parameters(self, lr):
        pass

class input_layer():

    def __init__(self, input_size):
        self.input_size = input_size

    def forward(self, x):
        return x

    def backward(self, error):
        return error
    
    def update_parameters(self, lr):
        pass

class flatten_layer():
    def __init__(self):
        pass

    def forward(self, x):
        # (bs, 28, 28) -> (bs, 784)
        return x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    
    def backward(self, error):
        pass
    def update_parameters(self, lr):
        pass
        
        

Model design

In [93]:
class NN_model():
    
    def __init__(self, input_layer_dimensions, hidden_layer_dimensions, output_layer_dimensions):
        self.sequential = [flatten_layer(),
                           input_layer(input_layer_dimensions), #Neurones d'entrada
                           linear_layer(input_layer_dimensions, hidden_layer_dimensions), #Redimensionar de 784 a 100
                           ReLu_layer(), #ReLu 
                           linear_layer(hidden_layer_dimensions, output_layer_dimensions)] #Redimensionar de 100 a 10
        
    def forward(self, x):
        for layer in self.sequential: #Cada layer del self.sequencial
            y = layer.forward(x)
            x = y #En la seguent capa la Y es converteix en la X i aixi succesivament.
        return y
    
    def backward(self, error):
        for layer in reversed(self.sequential): #Voltejem amb reversed ja que al backward va a l'inversa
            y = layer.backward(error)
            error = y #El mateix que el forward pero a la inversa
        return error
    
    def update_parameters(self, learning_rate = 0.03):
        for layer in self.sequential:
            layer.update_parameters(learning_rate)
        


model = NN_model(4,3,2)

x = np.array([[1,0,1,0],[0,1,1,1]])
ypred = model.forward(x)
error = np.array([[ 0.07912738,  0.77109688],[ 0.73532997, -0.13105195]])
model.backward(error)

IndexError: tuple index out of range

Train test loop

In [23]:
class Dataloader():

    def __init__(self, X, y, batch_size, shuffle = True):
        self.bs = batch_size

        randomizer = np.arange(len(y))
        # np.arange(len(y)) = np.array(range(len(y)))
        if shuffle:
            np.random.shuffle(randomizer)

        self.X = X[randomizer]
        self.y = y[randomizer]

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx]/255, self.y[idx]
    


    
    

    #idx.convert transforma en array com podras observar a baix
X_train = idx2numpy.convert_from_file('Mnist/train-images.idx3-ubyte')
y_train = idx2numpy.convert_from_file('Mnist/train-labels.idx1-ubyte')

X_test = idx2numpy.convert_from_file('Mnist/t10k-images.idx3-ubyte') 
y_test = idx2numpy.convert_from_file('Mnist/t10k-labels.idx1-ubyte') 
    
train_dataloader = Dataloader(X_train, y_train, 500)

In [24]:
prova, y = train_dataloader[5:2000:10]
print(prova.shape)

(200, 28, 28)


In [25]:
_, y = train_dataloader[0:5]
print(y)

def one_hot_encoding(y, num_classes):
    y_one_hot_encoded = np.zeros((len(y), num_classes))
    y_one_hot_encoded[np.arange(len(y)),y] = 1
    return y_one_hot_encoded

yr = one_hot_encoding(y,10)

[0 2 7 0 6]


## defining train steps and test loops

In [26]:
model = NN_model(784, 100, 10)

In [74]:
def train_step(model, X, y, lr, verbose = True):
    # X es un batch de dades

    logits = model.forward(X)
    yreal = one_hot_encoding(y, 10)
    error = -2 * (yreal - logits)
    model.backward(error)
    model.update_parameters(learning_rate=lr)
    cost = np.mean((logits - yreal) ** 2)
    if verbose:
        print("resultat funció de cost: ", cost)

def test_step(model, X, y, verbose = True):
    logits = model.forward(X)
    accuracy = np.sum(np.argmax(logits, axis=1) == y)/len(y)
    if verbose:
        print(accuracy)



X_batch, y_batch = train_dataloader[0:1000]
train_step(model, X_batch, y_batch, lr = 0.1)


resultat funció de cost:  0.1262632342340465


In [81]:
def train_loop(model, dataloader, lr, verbose):
    for i in np.arange(0, len(dataloader), dataloader.bs):
        X_batch, y_batch = dataloader[i:i+dataloader.bs]

        train_step(model, X_batch, y_batch, lr, verbose)

def test_loop(model, dataloader, verbose):
    for i in np.arange(0, len(dataloader), dataloader.bs):
        X_batch, y_batch = dataloader[i:i+dataloader.bs]

        test_step(model, X_batch, y_batch, verbose)

train_loop(model, train_dataloader, 0.03, True)


resultat funció de cost:  0.0899591089534486
resultat funció de cost:  0.08998027748094328
resultat funció de cost:  0.0899891807711429
resultat funció de cost:  0.08994684306920848
resultat funció de cost:  0.09001200964043042
resultat funció de cost:  0.08989444997834149
resultat funció de cost:  0.08989001133004565
resultat funció de cost:  0.08999743114356491
resultat funció de cost:  0.08995381243264879
resultat funció de cost:  0.09000704339847353
resultat funció de cost:  0.08999010912552492
resultat funció de cost:  0.08996604967583544
resultat funció de cost:  0.08997799221622611
resultat funció de cost:  0.08994226162501862
resultat funció de cost:  0.0899156423687265
resultat funció de cost:  0.09001166435447208
resultat funció de cost:  0.08999780016313978
resultat funció de cost:  0.0899703741907302
resultat funció de cost:  0.08997733462899274
resultat funció de cost:  0.09000757115633073
resultat funció de cost:  0.0899417028396558
resultat funció de cost:  0.08994183068

In [99]:
def main(hidden_size = 100, batch_size = 1000, learning_rate = 0.03, epochs = 10, verbose = True):

    X_train = idx2numpy.convert_from_file('Mnist/train-images.idx3-ubyte')
    y_train = idx2numpy.convert_from_file('Mnist/train-labels.idx1-ubyte')

    X_test = idx2numpy.convert_from_file('Mnist/t10k-images.idx3-ubyte') 
    y_test = idx2numpy.convert_from_file('Mnist/t10k-labels.idx1-ubyte') 
    
    train_dataloader = Dataloader(X_train, y_train, batch_size)
    test_dataloader = Dataloader(X_test, y_test, batch_size)
    model = NN_model(784, hidden_size, 10)
    

    for epoch in range(epochs):
        train_loop(model, train_dataloader, learning_rate, verbose)
        test_loop(model, test_dataloader,verbose=True)
    

In [100]:
main(hidden_size = 100,
     batch_size = 500,
     epochs = 10,
     verbose = False
     )

0.108
0.122
0.126
0.11
0.118
0.122
0.084
0.12
0.112
0.128
0.134
0.116
0.148
0.118
0.088
0.112
0.118
0.124
0.12
0.096
0.104
0.114
0.102
0.112
0.116
0.116
0.104
0.108
0.124
0.12
0.122
0.108
0.13
0.13
0.108
0.12
0.112
0.12
0.104
0.09
0.084
0.096
0.096
0.098
0.074
0.084
0.09
0.084
0.076
0.096
0.07
0.09
0.094
0.118
0.092
0.102
0.086
0.1
0.096
0.096
0.078
0.094
0.094
0.09
0.058
0.084
0.088
0.082
0.08
0.086
0.056
0.09
0.094
0.102
0.082
0.096
0.09
0.096
0.096
0.09
0.074
0.09
0.088
0.088
0.056
0.074
0.092
0.074
0.076
0.078
0.064
0.074
0.092
0.104
0.092
0.084
0.096
0.096
0.104
0.082
0.08
0.094
0.082
0.09
0.056
0.076
0.08
0.072
0.084
0.086
0.06
0.08
0.086
0.11
0.09
0.088
0.084
0.09
0.106
0.088
0.084
0.088
0.078
0.092
0.062
0.074
0.078
0.076
0.084
0.082
0.064
0.078
0.088
0.104
0.09
0.084
0.084
0.094
0.098
0.084
0.086
0.094
0.082
0.096
0.054
0.064
0.082
0.074
0.082
0.084
0.06
0.084
0.09
0.11
0.082
0.082
0.088
0.092
0.104
0.094
0.088
0.1
0.086
0.096
0.056
0.06
0.078
0.074
0.076
0.078
0.062
0.082
0.0